In [1]:
import os
os.getcwd()

'/home/agustin/plant-traits/notebooks'

In [2]:
cd ..

/home/agustin/plant-traits


/home/agustin/.cache/pypoetry/virtualenvs/plant-traits-y5eu6WKG-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import pandas as pd
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from src.preprocess_utils import drop_outliers, scale_variables
from src.constants import TARGETS, SD, ID, LOG_TARGETS

In [4]:
train_df = pd.read_csv('data/planttraits2024/train.csv')
test_df = pd.read_csv('data/planttraits2024/test.csv')

pd.Series(train_df.columns.sort_values()).to_csv('data/columns.csv')
variables = train_df.columns[~train_df.columns.isin(TARGETS+SD+[ID])]
meta_variables = variables.str.split('_').str[:2].str.join("_").unique()

In [5]:
train_df.shape

(55489, 176)

In [6]:
train_df.drop(axis=1, labels=TARGETS+SD).describe()

,id,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m03,VOD_X_1997_2018_multiyear_mean_m04,VOD_X_1997_2018_multiyear_mean_m05,VOD_X_1997_2018_multiyear_mean_m06,VOD_X_1997_2018_multiyear_mean_m07,VOD_X_1997_2018_multiyear_mean_m08,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12
count,5.548900e+04,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,...,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000,55489.000000
mean,1.714553e+08,14.577142,1058.710939,138.164359,56.727213,518.711405,25.227185,116.895673,140.259205,128.187695,...,0.374976,0.385622,0.391511,0.394641,0.396362,0.393833,0.391273,0.385420,0.378187,0.373244
std,4.087868e+07,7.584290,768.992166,136.409312,31.692061,316.618440,9.151237,17.607857,15.075214,16.030602,...,0.126546,0.130438,0.134787,0.140129,0.139662,0.138971,0.138265,0.134961,0.128688,0.126095
min,2.637500e+04,-13.731130,0.000000,0.000000,0.000000,9.877081,7.658674,29.000000,46.000000,42.000000,...,0.010382,0.008422,0.003341,0.005615,0.005608,0.008641,0.007320,0.007639,0.010804,0.010404
25%,1.658614e+08,9.247916,528.315552,53.000000,29.685472,300.478760,18.800001,106.000000,131.000000,118.000000,...,0.285188,0.292933,0.293150,0.291820,0.294253,0.290927,0.290144,0.287874,0.289432,0.287467
50%,1.915103e+08,15.536692,853.776184,95.500000,50.196312,446.522308,23.681778,118.000000,142.000000,129.000000,...,0.359904,0.372863,0.376943,0.376403,0.377365,0.375070,0.372533,0.368636,0.363573,0.359759
75%,1.951001e+08,20.238457,1352.000000,181.485718,80.811249,690.003845,30.399998,130.000000,150.000000,139.000000,...,0.461714,0.478076,0.488280,0.496335,0.496437,0.490374,0.487336,0.478870,0.465322,0.455181
max,1.967668e+08,30.294445,8392.463867,2448.964355,169.582291,2025.894653,63.657616,194.000000,192.000000,201.000000,...,0.784878,0.803896,0.798269,0.820608,0.869486,0.851053,0.836620,0.832753,0.805387,0.780904


In [7]:
description = train_df.drop(axis=1, labels=TARGETS+SD).apply(zscore).describe(percentiles=[.01, .25, .5, .75, .99]).drop(axis=1, labels=[ID])
description

,WORLDCLIM_BIO1_annual_mean_temperature,WORLDCLIM_BIO12_annual_precipitation,WORLDCLIM_BIO13.BIO14_delta_precipitation_of_wettest_and_dryest_month,WORLDCLIM_BIO15_precipitation_seasonality,WORLDCLIM_BIO4_temperature_seasonality,WORLDCLIM_BIO7_temperature_annual_range,SOIL_bdod_0.5cm_mean_0.01_deg,SOIL_bdod_100.200cm_mean_0.01_deg,SOIL_bdod_15.30cm_mean_0.01_deg,SOIL_bdod_30.60cm_mean_0.01_deg,...,VOD_X_1997_2018_multiyear_mean_m03,VOD_X_1997_2018_multiyear_mean_m04,VOD_X_1997_2018_multiyear_mean_m05,VOD_X_1997_2018_multiyear_mean_m06,VOD_X_1997_2018_multiyear_mean_m07,VOD_X_1997_2018_multiyear_mean_m08,VOD_X_1997_2018_multiyear_mean_m09,VOD_X_1997_2018_multiyear_mean_m10,VOD_X_1997_2018_multiyear_mean_m11,VOD_X_1997_2018_multiyear_mean_m12
count,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,...,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04,5.548900e+04
mean,1.659542e-16,-6.095232e-17,-1.703080e-17,-8.861136e-17,4.161661e-17,-1.289475e-16,2.422727e-16,1.920766e-16,1.618566e-16,1.160143e-16,...,-7.273302e-17,1.622407e-16,-1.883632e-16,-1.667225e-16,3.636651e-17,1.290755e-16,6.837929e-17,1.682591e-16,-1.765825e-16,1.735733e-16
std,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,...,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00,1.000009e+00
min,-3.732522e+00,-1.376764e+00,-1.012875e+00,-1.789966e+00,-1.607104e+00,-1.919814e+00,-4.991889e+00,-6.252651e+00,-5.376496e+00,-5.772360e+00,...,-2.881158e+00,-2.891829e+00,-2.879910e+00,-2.776226e+00,-2.797874e+00,-2.771760e+00,-2.776965e+00,-2.799203e+00,-2.854848e+00,-2.877533e+00
1%,-2.313099e+00,-1.312545e+00,-9.376210e-01,-1.457318e+00,-1.520722e+00,-1.806808e+00,-2.379395e+00,-2.537911e+00,-2.444577e+00,-2.633245e+00,...,-2.100116e+00,-2.206113e+00,-2.132757e+00,-2.037010e+00,-2.004184e+00,-2.035771e+00,-2.093807e+00,-2.105255e+00,-2.096676e+00,-2.125395e+00
25%,-7.026727e-01,-6.897342e-01,-6.243351e-01,-8.532731e-01,-6.892669e-01,-7.023359e-01,-6.188017e-01,-6.142061e-01,-6.355211e-01,-5.832107e-01,...,-7.095365e-01,-7.106079e-01,-7.297617e-01,-7.337701e-01,-7.311184e-01,-7.404927e-01,-7.314240e-01,-7.227750e-01,-6.896916e-01,-6.802582e-01
50%,1.265192e-01,-2.665002e-01,-3.127700e-01,-2.060756e-01,-2.280023e-01,-1.688757e-01,6.271842e-02,1.154751e-01,5.067261e-02,1.214886e-01,...,-1.191061e-01,-9.782107e-02,-1.080868e-01,-1.301518e-01,-1.360183e-01,-1.350160e-01,-1.355382e-01,-1.243583e-01,-1.135602e-01,-1.069473e-01
75%,7.464595e-01,3.813975e-01,3.175864e-01,7.599459e-01,5.410108e-01,5.652634e-01,7.442385e-01,6.461522e-01,6.744851e-01,6.339972e-01,...,6.854328e-01,7.087965e-01,7.179459e-01,7.257254e-01,7.165615e-01,6.946919e-01,6.947836e-01,6.924276e-01,6.771050e-01,6.498054e-01
99%,1.691435e+00,3.287396e+00,3.274400e+00,2.237373e+00,2.883345e+00,2.868434e+00,1.993692e+00,2.238184e+00,2.171635e+00,2.171523e+00,...,2.355885e+00,2.269686e+00,2.224611e+00,2.208540e+00,2.207530e+00,2.245610e+00,2.270964e+00,2.230789e+00,2.334971e+00,2.424951e+00
max,2.072369e+00,9.536923e+00,1.694035e+01,3.561021e+00,4.760294e+00,4.199517e+00,4.379012e+00,3.432207e+00,4.542123e+00,3.196540e+00,...,3.239195e+00,3.206717e+00,3.017811e+00,3.039851e+00,3.387669e+00,3.290065e+00,3.220998e+00,3.314566e+00,3.319680e+00,3.232976e+00


In [8]:
magnitude_ranges = {"X4_mean": (1e-5, 1e1),
                    "X11_mean": (1e-1, 1e2),
                    "X18_mean": (1e-2, 1e2),
                    "X26_mean": (1e-2, 1e4),
                    "X50_mean": (1e-1, 1e2),
                    "X3112_mean": (1e0, 1e5)
                    }
magnitude_mask = train_df[TARGETS].apply(lambda x: (x < magnitude_ranges[x.name][0]) | (x > magnitude_ranges[x.name][1]), axis=0)
print(f"We have {magnitude_mask.any(axis=1).sum()} magnitude outliers")

z = 2.5
remove_magnitude_outliers = True


wo_outliers_train_df, outliers = drop_outliers(train_df, z, apply_cols=TARGETS)
wo_outliers_train_df = wo_outliers_train_df.reset_index(drop=True)

#Drop magnitude outliers if still any
if remove_magnitude_outliers:
    wo_outliers_train_df = wo_outliers_train_df[~wo_outliers_train_df.index.isin(magnitude_mask[magnitude_mask.any(axis=1)].index)]

wo_outliers_merged_transformed_train_df, merged_transformed_test_df, scalers = scale_variables(wo_outliers_train_df, test_df, log_scale_targets=LOG_TARGETS)
# full_merged_transformed_train_df, _, scalers = scale_variables(train_df)

We have 825 magnitude outliers
Dropping 449 rows (75 outliers per column)


/home/agustin/plant-traits/src/preprocess_utils.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets_df[log_scale_targets] = targets_df[log_scale_targets].apply(np.log)


In [9]:
s = f'data/planttraits2024/transformed_train_df_{z}z_targets.csv'
test_s = f'data/planttraits2024/transformed_test_df_{z}z_targets.csv'
if remove_magnitude_outliers:
    s = f'data/planttraits2024/transformed_train_df_{z}z_wo_magnitude_ouliers_targets.csv'
    test_s = f'data/planttraits2024/transformed_test_df_{z}_wo_magnitude_ouliers_targets.csv'
    
wo_outliers_merged_transformed_train_df.to_csv(s, index=False)
merged_transformed_test_df.to_csv(test_s, index=False)

print(s)

scalers[1].to_csv('data/std_scaler.csv')


data/planttraits2024/transformed_train_df_2.5z_wo_magnitude_ouliers_targets.csv


In [13]:
sc = pd.read_csv('data/std_scaler.csv', index_col='targets')

In [10]:
target=TARGETS[0]

In [11]:
full_corr_df = full_merged_transformed_train_df.corr(method='spearman')
zscored_corr_df = wo_outliers_merged_transformed_train_df.corr(method='spearman')

NameError: name 'full_merged_transformed_train_df' is not defined

In [ ]:
l = []
l_nt = []
for target in TARGETS:
    partial0 = zscored_corr_df.loc[target].drop(labels=[target]).reset_index().reset_index().rename({'level_0': 'idx zscored'}, axis=1)
    partial1 = full_corr_df.loc[target].drop(labels=[target]).reset_index().reset_index().rename({'level_0': 'idx full'}, axis=1)

    partial0_no_targets = zscored_corr_df.loc[target].drop(labels=TARGETS).reset_index().reset_index().rename({'level_0': 'idx zscored'}, axis=1)
    partial1_no_targets = full_corr_df.loc[target].drop(labels=TARGETS).reset_index().reset_index().rename({'level_0': 'idx full'}, axis=1)

    
    partial0['abs'] = partial0[target].abs()
    partial1['abs'] = partial1[target].abs()
    
    partial0 = partial0.sort_values(by='abs', ascending=False).drop(axis=1, labels=['abs']).reset_index(drop=True)
    partial1 = partial1.sort_values(by='abs', ascending=False).drop(axis=1, labels=['abs']).reset_index(drop=True)
    
    l.extend([partial0, partial1])
    
    l_nt.extend([partial0[~partial0.loc[:, 'index'].isin(TARGETS)].reset_index(drop=True), partial1[~partial1.loc[:, 'index'].isin(TARGETS)].reset_index(drop=True)])

    
    
concat_df = pd.concat(l, axis=1)
concat_df.to_csv('data/concat.csv')
concat_df = pd.concat(l_nt, axis=1)
concat_df.to_csv('data/concat_no_targets.csv')
    

In [ ]:
group_var = meta_variables[0]

In [ ]:
for group_var in meta_variables:
    cols = full_merged_transformed_train_df.columns[full_merged_transformed_train_df.columns.str.contains(group_var)]
    full_merged_transformed_train_df[cols].corr().to_csv(f'data/{group_var}_corr.csv')
    triu = np.triu(full_merged_transformed_train_df[cols].corr()).flatten()
    print(f'{group_var}: mean cor {triu[(triu != 0) & (triu != 1.)].mean()}')


In [ ]:
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(wo_outliers_merged_transformed_train_df[variables], wo_outliers_merged_transformed_train_df[TARGETS], random_state=65465)
for target in TARGETS:
    
    ada_boost_regressor = AdaBoostRegressor(n_estimators=100)
    gradient_boost_regressor = GradientBoostingRegressor(max_depth=3, max_features='sqrt', n_estimators=100)
    y_train_t = y_train[target]
    y_test_t = y_test[target]
    ada_boost_regressor.fit(X_train, y_train_t)
    ada_boost_r2 = 1 - np.power(y_test_t-ada_boost_regressor.predict(X_test), 2).sum()/np.power(y_test_t-y_test_t.mean(), 2).sum()
    print(f"Adaboost Scores for {target}: train score: {ada_boost_regressor.score(X_train, y_train_t):2.4f}, test score: {ada_boost_regressor.score(X_test, y_test_t):2.4f}, R2: {ada_boost_r2:2.4f}")

    gradient_boost_regressor.fit(X_train, y_train_t)
    grad_boost_r2 = 1 - np.power(y_test_t-gradient_boost_regressor.predict(X_test), 2).sum()/np.power(y_test_t-y_test_t.mean(), 2).sum()
    print(f"Gradient boost Scores for {target}: train score: {gradient_boost_regressor.score(X_train, y_train_t):2.4f}, test score: {gradient_boost_regressor.score(X_test, y_test_t):2.4f}, R2: {grad_boost_r2:2.4f}\n")
    

    
